In [50]:
import os
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import pytesseract
import shutil
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

class ImageDatasetCleaner:
    def __init__(self, csv_path, image_folder, output_folder):
        """
        Inizializza il cleaner con il file CSV, la cartella delle immagini e la cartella di output
        """
        self.df = pd.read_csv(csv_path, header=None, names=["Image", "Label"])
        self.image_folder = image_folder
        self.output_folder = output_folder
        self.bad_images_folder = os.path.join(output_folder, "rejected")
        self.clean_images_folder = os.path.join(output_folder, "clean")
        
        # Crea le cartelle di output se non esistono
        for folder in [self.output_folder, self.bad_images_folder, self.clean_images_folder]:
            if not os.path.exists(folder):
                os.makedirs(folder)
        
        # Carica il modello face detection
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        # Crea un nuovo DataFrame per tenere traccia dei risultati
        self.results_df = self.df.copy()
        self.results_df['Status'] = 'clean'
        self.results_df['Rejection_Reason'] = ''

    def has_text(self, image_path, text_threshold=10):
        """
        Rileva la presenza di testo nell'immagine usando pytesseract
        """
        try:
            image = Image.open(image_path)
            text = pytesseract.image_to_string(image)
            # Conta il numero di caratteri alfanumerici
            char_count = sum(c.isalnum() for c in text)
            return char_count > text_threshold
        except Exception as e:
            print(f"Errore nel rilevamento del testo per {image_path}: {str(e)}")
            return False

    def has_faces(self, image_path, min_face_size=0.1, scale_factor=1.2, min_neighbors=8):
        """
        Rileva la presenza di volti nell'immagine con parametri ottimizzati
        per ridurre i falsi positivi
        
        Parameters:
        - min_face_size: dimensione minima del volto rispetto all'immagine (0.1 = 10% dell'immagine)
        - scale_factor: fattore di scala per la detection (più alto = più veloce ma meno preciso)
        - min_neighbors: numero minimo di detection vicine per considerare un volto (più alto = meno falsi positivi)
        """
        try:
            image = cv2.imread(image_path)
            if image is None:
                return False
                
            height, width = image.shape[:2]
            min_size = int(min(height, width) * min_face_size)
            
            # Converti in scala di grigi
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Equalizza l'istogramma per migliorare il contrasto
            gray = cv2.equalizeHist(gray)
            
            # Rileva volti frontali
            faces_front = self.face_cascade.detectMultiScale(
                gray,
                scaleFactor=scale_factor,
                minNeighbors=min_neighbors,
                minSize=(min_size, min_size)
            )
        
            
            # Verifica la presenza di volti di dimensione significativa
            all_faces = list(faces_front)
            
            for (x, y, w, h) in all_faces:
                # Calcola l'area relativa del volto rispetto all'immagine
                face_area = w * h
                image_area = height * width
                face_ratio = face_area / image_area
                
                # Se troviamo almeno un volto di dimensione significativa, restituisci True
                if face_ratio > min_face_size * min_face_size:
                    return True
            
            return False
            
        except Exception as e:
            print(f"Errore nel rilevamento dei volti per {image_path}: {str(e)}")
            return False
        
        
    def is_cartoon(self, image_path):
        """
        Riconosce se un'immagine ha caratteristiche di un cartone animato.
        
        Args:
            image_path (str): Percorso dell'immagine da analizzare.
        
        Returns:
            bool: True se l'immagine è un cartone, False altrimenti.
        """
        try:
            # Carica l'immagine
            image = cv2.imread(image_path)
            if image is None:
                print(f"Impossibile caricare l'immagine: {image_path}")
                return False
            
            # Converti l'immagine in scala di grigi
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            # Rileva i bordi con l'algoritmo di Canny
            edges = cv2.Canny(gray, threshold1=100, threshold2=200)
            
            # Conta i contorni
            contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contour_count = len(contours)
            
            # Calcola la saturazione media
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            saturation = hsv[:, :, 1]  # Canale della saturazione
            avg_saturation = np.mean(saturation)
            
            # Calcola la deviazione standard dei colori
            color_std = np.std(image)
        
            # Heuristica per determinare se è un cartone:
            # - Molti contorni (indicando bordi definiti)
            # - Alta saturazione media (indicando colori vividi)
            # - Bassa deviazione standard dei colori (indicando colori piatti)
            is_cartoon = contour_count > 1000 and avg_saturation > 50 and color_std < 70
            
            return is_cartoon
    
        except Exception as e:
            print(f"Errore nel rilevamento cartoon per {image_path}: {str(e)}")
            return False


    def clean_dataset(self):
        """
        Pulisce il dataset spostando le immagini nelle appropriate cartelle e aggiorna il CSV
        """
        total_images = len(self.df)
        rejected_images = 0
        
        for idx, row in self.df.iterrows():
            image_path = os.path.join(self.image_folder, row['Image'])
            
            if not os.path.exists(image_path):
                print(f"Immagine non trovata: {image_path}")
                continue
            
            should_reject = False
            reasons = []
            
            # Controlla testo
            # if self.has_text(image_path):
            #     should_reject = True
            #     reasons.append("testo")
            
            # # # Controlla volti
            # if self.has_faces(image_path):
            #     should_reject = True
            #     reasons.append("volti")
            
            # Controlla se è un cartone
            if self.is_cartoon(image_path):
                should_reject = True
                reasons.append("cartone animato")
            
            # Aggiorna il DataFrame dei risultati
            if should_reject:
                rejected_images += 1
                self.results_df.at[idx, 'Status'] = 'rejected'
                self.results_df.at[idx, 'Rejection_Reason'] = ', '.join(reasons)
                dest_path = os.path.join(self.bad_images_folder, row['Image'])
                print(f"Rifiutato {row['Image']} per: {', '.join(reasons)}")
            else:
                dest_path = os.path.join(self.clean_images_folder, row['Image'])
            
            # Copia l'immagine nella cartella appropriata
            shutil.copy2(image_path, dest_path)
        
        # Salva il DataFrame aggiornato
        self.results_df.to_csv(os.path.join(self.output_folder, 'cleaned_dataset.csv'), index=False)
        
        return total_images, rejected_images

def main():
    # Esempio di utilizzo con la struttura del dataset fornita
    csv_path = 'dataset/train_small.csv'
    image_folder = 'dataset/train_set'
    output_folder = 'dataset/cleaned_output'
    
    cleaner = ImageDatasetCleaner(csv_path, image_folder, output_folder)
    total, rejected = cleaner.clean_dataset()
    
    print(f"\nRisultati pulizia dataset:")
    print(f"Totale immagini processate: {total}")
    print(f"Immagini rifiutate: {rejected}")
    print(f"Percentuale rifiutate: {(rejected/total)*100:.2f}%")

if __name__ == "__main__":
    main()

Rifiutato train_067490.jpg per: cartone animato
Rifiutato train_074552.jpg per: cartone animato
Rifiutato train_032804.jpg per: cartone animato
Rifiutato train_018005.jpg per: cartone animato
Rifiutato train_026961.jpg per: cartone animato
Rifiutato train_094889.jpg per: cartone animato
Rifiutato train_031690.jpg per: cartone animato
Rifiutato train_031699.jpg per: cartone animato
Rifiutato train_098586.jpg per: cartone animato
Rifiutato train_016284.jpg per: cartone animato
Rifiutato train_062919.jpg per: cartone animato
Rifiutato train_032351.jpg per: cartone animato
Rifiutato train_007933.jpg per: cartone animato
Rifiutato train_081767.jpg per: cartone animato
Rifiutato train_082015.jpg per: cartone animato
Rifiutato train_036198.jpg per: cartone animato
Rifiutato train_020751.jpg per: cartone animato
Rifiutato train_038184.jpg per: cartone animato
Rifiutato train_013142.jpg per: cartone animato
Rifiutato train_013336.jpg per: cartone animato
Rifiutato train_063942.jpg per: cartone 

In [21]:
import pandas as pd
import numpy as np
import cv2
import os
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.covariance import EllipticEnvelope

class ImageDatasetCleaner:
    def __init__(self, csv_path, image_folder, output_folder):
        """
        Inizializza il cleaner con il file CSV, la cartella delle immagini e la cartella di output
        """
        self.df = pd.read_csv(csv_path, header=None, names=["Image", "Label"])
        self.image_folder = image_folder
        self.output_folder = output_folder
        self.bad_images_folder = os.path.join(output_folder, "rejected")
        self.clean_images_folder = os.path.join(output_folder, "clean")
        
        # Crea le cartelle di output se non esistono
        for folder in [self.output_folder, self.bad_images_folder, self.clean_images_folder]:
            if not os.path.exists(folder):
                os.makedirs(folder)
        
        # Carica il modello face detection
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        # Crea un nuovo DataFrame per tenere traccia dei risultati
        self.results_df = self.df.copy()
        self.results_df['Status'] = 'clean'
        self.results_df['Rejection_Reason'] = ''
        
        # Parametri LBP
        self.n_points = 24
        self.radius = 8
        self.target_size = (256, 256)  # Dimensione standard per tutte le immagini
        
    def extract_lbp_features(self, image_path):
        """
        Estrae le feature LBP da un'immagine
        """
        try:
            # Carica e ridimensiona l'immagine
            img = cv2.imread(image_path)
            if img is None:
                return None
                
            img = cv2.resize(img, self.target_size)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # Calcola LBP
            lbp = local_binary_pattern(gray, self.n_points, self.radius, method='uniform')
            
            # Calcola l'istogramma delle feature LBP
            n_bins = self.n_points + 2
            hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
            
            return hist
            
        except Exception as e:
            print(f"Errore nell'elaborazione di {image_path}: {str(e)}")
            return None
            
    def detect_outliers(self, contamination=0.1):
        """
        Rileva gli outlier usando le feature LBP
        """
        print("Estraendo le feature LBP...")
        
        # Estrai le feature per tutte le immagini
        features_list = []
        valid_indices = []
        
        for idx, row in self.df.iterrows():
            image_path = os.path.join(self.image_folder, row['Image'])
            features = self.extract_lbp_features(image_path)
            
            if features is not None:
                features_list.append(features)
                valid_indices.append(idx)
        
        if not features_list:
            raise ValueError("Nessuna feature valida estratta dalle immagini")
            
        # Converti in array numpy
        X = np.array(features_list)
        
        # Standardizza le feature
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Applica PCA per ridurre la dimensionalità (opzionale ma consigliato)
        pca = PCA(n_components=0.95)  # Mantiene il 95% della varianza
        X_pca = pca.fit_transform(X_scaled)
        
        # Usa Elliptic Envelope per il rilevamento degli outlier
        outlier_detector = EllipticEnvelope(contamination=contamination, random_state=42)
        labels = outlier_detector.fit_predict(X_pca)
        
        # Aggiorna il DataFrame dei risultati
        for idx, label in zip(valid_indices, labels):
            if label == -1:  # outlier
                self.results_df.at[idx, 'Status'] = 'rejected'
                self.results_df.at[idx, 'Rejection_Reason'] = 'LBP outlier'
                
        # Calcola e stampa le statistiche
        n_outliers = sum(labels == -1)
        print(f"Trovati {n_outliers} outlier su {len(labels)} immagini totali")
        
    def process_dataset(self, contamination=0.1):
        """
        Elabora l'intero dataset applicando il rilevamento degli outlier
        """
        # Rileva gli outlier
        self.detect_outliers(contamination)
        
        # Sposta le immagini nelle cartelle appropriate
        for idx, row in self.results_df.iterrows():
            src_path = os.path.join(self.image_folder, row['Image'])
            
            if row['Status'] == 'rejected':
                dst_path = os.path.join(self.bad_images_folder, row['Image'])
            else:
                dst_path = os.path.join(self.clean_images_folder, row['Image'])
                
            try:
                # Copia l'immagine nella cartella appropriata
                import shutil
                shutil.copy2(src_path, dst_path)
            except Exception as e:
                print(f"Errore nel copiare {src_path}: {str(e)}")
        
        # Salva i risultati in un nuovo CSV
        results_path = os.path.join(self.output_folder, 'cleaning_results.csv')
        self.results_df.to_csv(results_path, index=False)
        
        print(f"Elaborazione completata. Risultati salvati in {results_path}")

In [22]:
cleaner = ImageDatasetCleaner('dataset/train_small.csv', 'dataset/train_set', 'output_folder')
cleaner.process_dataset(contamination=0.1)  # contamination è la percentuale attesa di outlier

Estraendo le feature LBP...
Trovati 502 outlier su 5020 immagini totali
Errore nel copiare dataset/train_set\train_059371.jpg: [WinError 3] The system cannot find the path specified
Errore nel copiare dataset/train_set\train_059389.jpg: [WinError 3] The system cannot find the path specified
Errore nel copiare dataset/train_set\train_059425.jpg: [WinError 3] The system cannot find the path specified
Errore nel copiare dataset/train_set\train_059445.jpg: [WinError 3] The system cannot find the path specified
Errore nel copiare dataset/train_set\train_059516.jpg: [WinError 3] The system cannot find the path specified
Errore nel copiare dataset/train_set\train_059538.jpg: [WinError 3] The system cannot find the path specified
Errore nel copiare dataset/train_set\train_059541.jpg: [WinError 3] The system cannot find the path specified
Errore nel copiare dataset/train_set\train_059570.jpg: [WinError 3] The system cannot find the path specified
Errore nel copiare dataset/train_set\train_05971

OSError: Cannot save file into a non-existent directory: 'output_folder'

In [7]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from torchvision import transforms, models
from PIL import Image
import torch

class ImageDatasetCleaner:
    def __init__(self, csv_path, image_folder, output_folder):
        """
        Inizializza il cleaner con il file CSV, la cartella delle immagini e la cartella di output.
        """
        self.df = pd.read_csv(csv_path, header=None, names=["Image", "Label"])
        self.image_folder = image_folder
        self.output_folder = output_folder
        self.bad_images_folder = os.path.join(output_folder, "rejected")
        self.clean_images_folder = os.path.join(output_folder, "clean")

        # Crea le cartelle di output se non esistono
        for folder in [self.output_folder, self.bad_images_folder, self.clean_images_folder]:
            if not os.path.exists(folder):
                os.makedirs(folder)

        # Modello pre-addestrato
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = models.resnet18(pretrained=True)
        self.model = torch.nn.Sequential(*(list(self.model.children())[:-1]))
        self.model.to(self.device)
        self.model.eval()

    def extract_features(self, image_path):
        """
        Estrae i feature embeddings per un'immagine.
        """
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
        image = Image.open(image_path).convert("RGB")
        image_tensor = transform(image).unsqueeze(0).to(self.device)

        with torch.no_grad():
            features = self.model(image_tensor).squeeze().cpu().numpy()
        return features

    def find_outliers(self):
        """
        Identifica le immagini outlier basandosi sui feature embeddings.
        """
        feature_list = []
        image_paths = []

        # Estrai feature per ogni immagine
        for _, row in self.df.iterrows():
            image_name = row["Image"]
            image_path = os.path.join(self.image_folder, image_name)

            if os.path.exists(image_path):
                try:
                    features = self.extract_features(image_path)
                    feature_list.append(features)
                    image_paths.append(image_name)
                except Exception as e:
                    print(f"Errore nell'elaborare l'immagine {image_name}: {e}")
                    continue

        # Converte in array per analisi
        feature_array = np.array(feature_list)

        # Modello per rilevare outlier
        clf = IsolationForest(contamination=0.05, random_state=42)
        outlier_pred = clf.fit_predict(feature_array)

        # Salva risultati
        self.df["Outlier"] = [1 if pred == -1 else 0 for pred in outlier_pred]
        self.df.to_csv(os.path.join(self.output_folder, "results.csv"), index=False)

        # Sposta le immagini nelle cartelle corrette
        for image_name, is_outlier in zip(image_paths, self.df["Outlier"]):
            src = os.path.join(self.image_folder, image_name)
            dest_folder = self.bad_images_folder if is_outlier else self.clean_images_folder
            dest = os.path.join(dest_folder, image_name)

            os.rename(src, dest)

        print("Pulizia completata. Risultati salvati in:", self.output_folder)


In [8]:
# Sostituisci i percorsi con i tuoi
cleaner = ImageDatasetCleaner('dataset/train_small.csv', 'dataset/train_set', 'output_folder3')
cleaner.find_outliers()

c:\Users\sguid\Uni\visual\project\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sguid\Uni\visual\project\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.